# Freebayes
Firstly, let's prepare the VCF that we want to filter. For this we need to do a series of things for the training of the ML model and for applying the model

## Training the model
For this, we use NA28178 from GIAB (https://jimb.stanford.edu/giab) as the truth set and we will examine the sites in our call set across all the chromosomes overlapping with the GIAB sites and consider them as true positives.

Firstly, we need to get the variant sites for NA12878 from our call set:

From here downwards, the process will be different in SNVs and INDELs

### SNVs

* Normalization of SNVs  
This is necessary in order to make a proper comparison with GIAB

For normalizaton I will use my Nextflow pipeline executed with the following command

* Now we train the model using only variants in chr20 that are in high-conf regions from GIAB

Let's select the variants in chr20:

And now, let's use for training the variants in the GIAB high-conf regions:

#### RFE
The Recursive Feature Elimination will help to select the variant annotations that are more useful in order the select the real SNVs:

We will select 25 features out of the X annotations that Freebayes generates:

And the selected features are:

And now we can run the Nextflow pipeline for training the model using GIAB NA12878:

The score of the model is: [LibLinear]Score for the logistic regression fitted model is: 0.9838855694369002

### INDELs

* Normalization of INDELs  
This is necessary in order to make a proper comparison with GIAB

For normalizaton I will use my Nextflow pipeline executed with the following command

#### RFE
The Recursive Feature Elimination will help to select the variant annotations that are more useful in order the select the real INDELs: In this case we use the INDELs from all the chros, as chr20 does not have enough INDELs

We will select 25 features out of the X annotations that Freebayes generates:

And the selected features are:

And now we can run the Nextflow pipeline for training the model using GIAB NA12878:

[LibLinear]Score for the logistic regression fitted model is: 0.7593647714874531

## Applying the model
This section relates how to apply the fitted logistic regression model that has been generated in the `Training the model` section

### chr20

First thing is to select the chr20 from the unfiltered call set:

In order to fix a problem with the VCF header causing bcftools to crash with error:

We need to rehead the VCF:

#### SNVs
We need to normalize the VCF that will be filtered.
In order to check that the process is working I will use only the chr20

And then we can use the normalization pipeline:

And then we can use the Nextflow pipeline with the fitted model produced in the `Training the model` section in order to apply the filtering on chr20:

* Assessing the quality of the filtered and unfiltered VCF with GIAB NA12878:

**PREFILTER:**

**POSTFILTER:**

**FILTERED with HARD_QUAL_CUTOFF**  
This section is on GIAB NA12878 benchmarking vs chr20 SNP Freebayes VCF filtered using just a hard quality cutoff=1

First, let's get the variants for chr20:

**QC:**

| Dataset       | shared (TP)  | giab_only (FN)| Igsr_only (FP) | total in giab | total in igsr |
| ------------- | ------------ | ------------- | -------------- | ------------- | ------------- |
| *Prefilter*   | 63,746 (95.2%)| 3,204 (4.8%) | 6,524 (9.3%)   | 66,950 (100%) | 70,270 (100%) |
| *Postfilter (c:0.950)* |59,717 (89.2%)  | 7,233 (10.8%) |5,753 (8.8%) | 66,950 (100%) | 65,470 (100%)|
| *Postfilter (c:0.955)* |59,713 (89.2%)  | 7,237 (10.8%) | 5,681 (8.7%)| 66,950 (100%) | 65,394 (100%)|
| *Postfilter (c:0.965)* |59,706 (89.2%)  |7,244 (10.8%)| 5,519 (8.5%) | 66,950 (100%) | 65,225 (100%) |
| *Postfilter (c:0.970)* |59,701 (89.2%)| 7,249 (10.8%) | 5,413 (8.3%) | 66,950 (100%) | 65,114 (100%) |
| *Postfilter (c:0.975)* |59,696 (89.2%)|7,254 (10.8%)| 5,266 (8.1%) | 66,950 (100%) | 64,962 (100%) |
| *Postfilter (c:0.980)* |59,677 (89.1%)| 7,273 (10.9%) | 5,095 (7.9%) | 66,950 (100%) | 64,772 (100%) |
| *Postfilter (c:0.985)* |59,663 (89.1%)|7,287 (10.9%) | 4,871 (7.5%) | 66,950 (100%) |  64,534 (100%) |
| *Postfilter (c:0.990)* |59,632 (89.1%)|7,318 (10.9%)| 4513 (7.0%) | 66,950 (100%) | 64,145 (100%) |
| *Postfilter (c:0.991)* |59,624 (89.1%)|7,326 (10.9%)|4,420 (6.9%) | 66,950 (100%) | 64,044 (100%) |
| *Postfilter (c:0.992)* |59,617 (89.0%)|7,333 (11.0%)|4,285 (6.7%) | 66,950 (100%) | 63,902 (100%) |
| *Postfilter (c:0.993)* |59,611 (89.0%)|7,339 (11.0%)| 4,130 (6.5%)| 66,950 (100%) | 63,741 (100%) |
| *Postfilter (c:0.994)* |59,594 (89.0%)|7,356 (11.0%)|3,933 (6.2%)| 66,950 (100%) | 63,527 (100%) |
| *Postfilter (c:0.995)* |59,575 (89.0%)|7,375 (11.0%)|3,683 (5.8%)| 66,950 (100%) | 63,258 (100%) |
| *Postfilter (c:0.996)* |59,548 (88.9%)|7,402 (11.1%)|3,303 (5.3%)| 66,950 (100%) | 62,851 (100%) |
| *Postfilter (c:0.997)* |59,526 (88.9%)|7,424 (11.1%)|2,785 (4.5%)| 66,950 (100%) | 62,311 (100%) |
| *Postfilter (c:0.998)* |59,454 (88.8%)|7,496 (11.2%)|2,021 (3.3%)| 66,950 (100%) | 61,475 (100%) |
| **Postfilter (c:0.999)** |**59,302 (88.6%)**|**7,648 (11.4%)**|**1,266 (2.1%)**| 66,950 (100%) | 60,568 (100%) |
| *Postfilter (c:1.00)* |9,941 (14.8%) | 57,009 (85.1%) |24 (0.2%) | 66,950 (100%) | 9965 (100%) |
| *QUAL_cutoff* |57,240 (85.5%) | 9,710 (14.5%) | 1,892 (3.2%) | 66,950 (100%) | 59,132 (100%) |

#### INDELs
We need to normalize the VCF that will be filtered.
In order to check that the process is working I will use only the chr20

And then we can use the normalization pipeline:

And then we can use the Nextflow pipeline with the fitted model produced in the `Training the model` section in order to apply the filtering on chr20:

* Assessing the quality of the filtered and unfiltered VCF with GIAB NA12878:

**PREFILTER:**

**POSTFILTER:**

**QC:**

| Dataset       | shared (TP)  | giab_only (FN)| Igsr_only (FP) | total in giab | total in igsr|
| ------------- | ------------ | ------------- | -------------- | ------------- | -------------|
| *Prefilter*   |6,920 (63.3%) |4,015 (36.7%)  |1,863 (21.2%)   | 10,935 (100%) | 8,783 (100%) |
| *Postfilter (c:0.1)* | 6,413 (58.6%)|4,522 (41.3%)|1,409 (18.0%)| 10,935 (100%) |  7,822(100%)|
| *Postfilter (c:0.2)* | 6,044 (55.3%)|4,891 (44.7%)|1,201 (16.6%) | 10,935 (100%) | 7,245 (100%)|
| *Postfilter (c:0.3)* | 5,757 (52.6%)|5,178 (47.3%)|924 (13.8%) | 10,935 (100%) | 6,681 (100%)|
| *Postfilter (c:0.4)* | 5,522 (50.5%) |5,413 (49.5%)|656 (10.6%) | 10,935 (100%) | 6,178 (100%)|
| *Postfilter (c:0.5)* | 5,345 (48.9%) |5,590 (51.1%)|428 (7.4%) | 10,935 (100%) | 5,773 (100%)|
| *Postfilter (c:0.6)* | 5,152 (47.1%) |5,783 (52.9%) |290 (5.3%) | 10,935 (100%) | 5,442 (100%)|
| *Postfilter (c:0.7)* | 4,955 (45.3%) |5,980 (54.7%) | 210 (4.1%)| 10,935 (100%) | 5,165 (100%)|
| *Postfilter (c:0.8)* | 4,640 (42.4%) |6,295 (57.6%) | 130 (2.7%)| 10,935 (100%) | 4,770 (100%)|
| *Postfilter (c:0.9)* | 3,920 (35.8%) |7,015 (64.2%) |62 (1.6%) | 10,935 (100%) | 3,982 (100%)|
| *Postfilter (c:1.0)* | 9 (0.1%)|10,926 (99.9%) |0 (0%) | 10,935 (100%) | 9 (100%)|
| *QUAL_cutoff* |4,343 (39.7%) | 6,592 (60.3%) |189 (4.2%) | 10,935 (100%) | 4,532 (100%) |

**FILTERED with HARD_QUAL_CUTOFF**  
This section is on GIAB NA12878 benchmarking vs chr20 INDEL Freebayes VCF filtered using just a hard quality cutoff=1

First, let's get the variants for chr20: